In [1]:
import pandas as pd 
import numpy as np
import re
import codecs
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords 
nltk.download('stopwords')


from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda, Dropout, Bidirectional
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import keras


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saraltayal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [2]:
TRAIN_CSV = './train.csv'
glove = './glove.6B.100d'
# MODEL_SAVING_DIR = './'

In [3]:
train_df = pd.read_csv(TRAIN_CSV)
train_df.dropna()
np.shape(train_df)
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
def load_glove_model(glove_file):

    f = codecs.open(glove_file + ".txt", 'r', encoding='utf-8')
    model = {}
    for line in f:
        split_line = line.split()
        word = split_line[0]
        embedding = np.array([float(val) for val in split_line[1:]])
        model[word] = embedding
    return model
def get_w2v(sentence, model):
    return np.array([model.get(val, np.zeros(100)) for val in sentence.split()], dtype=np.float64)

#build the model-> a bit slow ~30seconds 
model = load_glove_model(glove)


In [5]:
def preprocess(input):
    if type(input) != type("string"):
        print(input)
        return "skip" 
    input = input.lower()
    input = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", input)
    input = re.sub(r"what's", "what is ", input)
    input = re.sub(r"\'s", " ", input)
    input = re.sub(r"\'ve", " have ", input)
    input = re.sub(r"can't", "cannot ", input)
    input = re.sub(r"n't", " not ", input)
    input = re.sub(r"i'm", "i am ", input)
    input = re.sub(r"\'re", " are ", input)
    input = re.sub(r"\'d", " would ", input)
    input = re.sub(r"\'ll", " will ", input)
    input = re.sub(r",", " ", input)
    input = re.sub(r"\.", " ", input)
    input = re.sub(r"!", " ! ", input)
    input = re.sub(r"\/", " ", input)
    input = re.sub(r"\^", " ^ ", input)
    input = re.sub(r"\+", " + ", input)
    input = re.sub(r"\-", " - ", input)
    input = re.sub(r"\=", " = ", input)
    input = re.sub(r"'", " ", input)
    input = re.sub(r"(\d+)(k)", r"\g<1>000", input)
    input = re.sub(r":", " : ", input)
    input = re.sub(r" e g ", " eg ", input)
    input = re.sub(r" b g ", " bg ", input)
    input = re.sub(r" u s ", " american ", input)
    input = re.sub(r"\0s", "0", input)
    input = re.sub(r" 9 11 ", "911", input)
    input = re.sub(r"e - mail", "email", input)
    input = re.sub(r"j k", "jk", input)
    input = re.sub(r"\s{2,}", " ", input)
    return input

In [6]:
train_df = pd.read_csv(TRAIN_CSV)

indexesToRemove = []

lenTrain = np.shape(train_df['question1'])[0]

for col in ['question1', 'question2']:
    for row in tqdm(range(40000)): #lenTrain
        processedText = preprocess(train_df[col][row])
        if processedText == "skip":
            indexesToRemove.append(row)
        train_df.at[row,col] = get_w2v(processedText, model)

print(indexesToRemove)
print(np.shape(train_df['question1'])[0])
train_df = train_df.drop(indexesToRemove)
print(np.shape(train_df['question1'])[0])

100%|██████████| 40000/40000 [00:04<00:00, 9923.16it/s]
[]
404290
404290


In [7]:
left = train_df['question1'][0:40000].values
right = train_df['question2'][0:40000].values
Y = train_df['is_duplicate'][0:40000].values

maxlen = 35
indextracker = []

for index in range(len(right)):
    if len(left[index]) > maxlen or len(right[index]) > maxlen or len(left[index]) < 3 or len(right[index]) < 3:
        indextracker.append(index)
left = np.delete(left, indextracker)
right = np.delete(right, indextracker)
Y = np.delete(Y, indextracker)
assert len(left) == len(right) and len(left) == len(Y)
print(len(right))

#probably need to delete variables etc for memory lol


39506


In [8]:
def padZeros(input, maxLen):
    if np.shape(input)[0] == maxLen:
        return input
    lenToPad = maxLen - np.shape(input)[0]
    output = np.concatenate((input, np.zeros((lenToPad, 100))))
    return output
    return np.transpose(output)


In [9]:
left = [padZeros(elem, maxlen) for elem in left]
right = [padZeros(elem, maxlen) for elem in right]

left = np.reshape(left, (np.shape(left)[0], maxlen, 100)) #100 comes from the encoding
right = np.reshape(right, (np.shape(right)[0], maxlen, 100)) #100 comes from the encoding
Y = np.reshape(Y, (np.shape(Y)[0], 1))

In [10]:
print(np.shape(Y))
print(np.shape(left))
print(np.shape(right))


(39506, 1)
(39506, 35, 100)
(39506, 35, 100)


In [11]:
idx = np.random.randint(len(right))
print(left[idx])
print(right[idx])
print(Y[idx])
print(np.shape(Y))

[[-0.51533   0.83186   0.22457  ... -1.2024    1.1304    0.3479  ]
 [-0.084215  0.69597   0.28383  ... -0.54      0.58031   0.46698 ]
 [-0.23676   0.15659   0.30243  ... -0.16095   0.75331   0.29294 ]
 ...
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]]
[[ 0.30449  -0.19628   0.20225  ... -0.18385  -0.12432   0.27468 ]
 [ 0.77415  -0.18802  -0.14085  ... -0.15508  -0.84373   0.28243 ]
 [-0.11819   0.012215 -0.065054 ... -0.044096  0.40568  -0.52475 ]
 ...
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]]
[1]
(39506, 1)


In [12]:
# Model variables
n_hidden = 80
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 8

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=np.shape(left[0]), dtype='float32')
right_input = Input(shape=np.shape(right[0]), dtype='float32')



# Since this is a siamese network, both sides share the same LSTM
shared_lstm = Bidirectional(LSTM(n_hidden, return_sequences=True))
shared_lstm2 = LSTM(n_hidden)

left_output = shared_lstm(left_input)
right_output = shared_lstm(right_input)

left_output = Dropout(0.1)(left_output)
left_output = shared_lstm2(left_output)

right_output = Dropout(0.1)(right_output)
right_output = shared_lstm2(right_output)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adam(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

# Start training

malstm_trained = malstm.fit([left, right], Y, batch_size=batch_size, nb_epoch=n_epoch, validation_split = 0.2)



Train on 31604 samples, validate on 7902 samples
Epoch 1/8
31604/31604 [==============================] - 49s 2ms/step - loss: 0.2128 - accuracy: 0.6564 - val_loss: 0.2171 - val_accuracy: 0.6592
Epoch 2/8
31604/31604 [==============================] - 48s 2ms/step - loss: 0.1910 - accuracy: 0.7042 - val_loss: 0.1905 - val_accuracy: 0.7168
Epoch 3/8
31604/31604 [==============================] - 68s 2ms/step - loss: 0.1800 - accuracy: 0.7260 - val_loss: 0.1804 - val_accuracy: 0.7287
Epoch 4/8
31604/31604 [==============================] - 46s 1ms/step - loss: 0.1709 - accuracy: 0.7468 - val_loss: 0.1796 - val_accuracy: 0.7269
Epoch 5/8
31604/31604 [==============================] - 47s 1ms/step - loss: 0.1616 - accuracy: 0.7633 - val_loss: 0.1767 - val_accuracy: 0.7351
Epoch 6/8
31604/31604 [==============================] - 50s 2ms/step - loss: 0.1529 - accuracy: 0.7815 - val_loss: 0.1702 - val_accuracy: 0.7483
Epoch 7/8
31604/31604 [==============================] - 57s 2ms/step - los

In [13]:
def predict(inputA, inputB, model = model):
    processedTextA = preprocess(inputA)
    processedTextB = preprocess(inputB)
    if processedTextA == "skip" or processedTextB == "skip":
        print("Input needs to be strings")
        return
    vectorizedA = get_w2v(processedTextA, model)
    vectorizedB = get_w2v(processedTextB, model)
    paddedA = padZeros(vectorizedA, maxlen)
    paddedB = padZeros(vectorizedB, maxlen)

    prediction = malstm.predict([[paddedA], [paddedB]])
    return prediction

In [15]:
print(predict("I like to eat", "I love to eat"))
print(predict("Sally went down the road", "Down the road Sally went"))
print(predict("I like to eat", "I hate to eat"))
print(predict("I enjoy to eat", "I love to eat"))
print(predict("What can make Physics easy to learn?", "How can you make physics easy to learn?"))
print(predict("This is the statement the Chief Minister gave in the floor of the assembly, which essentially is a fact.", "How can a statement the Chief Minister gave in the floor of the assembly, which essentially is a fact, be characterized as a claim?"))

[[0.29617628]]
[[0.5939616]]
[[0.21363927]]
[[0.37795046]]
[[0.83696353]]
[[0.38213742]]


In [ ]:
#tricks to try- attention layer
#Triplet loss